In [ ]:
import shutil
import sys
import time
from os.path import join
from substrateinterface import SubstrateInterface, Keypair

In [ ]:
alephhome = '/home/hansu/aleph/aleph-node/'
sys.path.append(join(alephhome, 'local-tests'))
from chainrunner import Chain, Seq, generate_keys

In [ ]:
testhome = '/home/hansu/aleph/desk'
# Path to working directory, where chainspec, logs and nodes' dbs are written:
workdir = join(testhome, 'workdir')
# Path to the pre-update aleph-node binary:
binary = join(testhome, 'bin', 'aleph-node-10.1')
# Path to the post-update aleph-node binary:
newbin = join(testhome, 'bin', 'aleph-node-11.1')
# Path to the post-update compiled runtime:
runtime = join(testhome, 'bin','aleph_runtime.64')

In [ ]:
N_VALIDATORS = 7 #at most 10, unless you extend names list below
# Ports for node 0, consecutive numbers are used for other nodes 
PORT = 30334
WS_PORT = 9943
RPC_PORT = 9933
VAL_PORT = 30343

In [ ]:
names = ['//Alice','//Bob','//Charlie','//Dave','//Eve','//Ferdie','//George','//Hans','//Iris','//James']
phrases = names[:N_VALIDATORS]
keys = generate_keys(binary, phrases)

sudo = keys['//Alice']
faucet = keys['//Bob']
val_addrs = ['127.0.0.1:'+str(i) for i in range(VAL_PORT, VAL_PORT+N_VALIDATORS)]

shutil.rmtree(workdir, True)
chain = Chain(workdir)
chain.bootstrap(binary, keys.values(),
                chain_type='live',
                faucet_account_id=faucet,
                sudo_account_id=sudo,
                raw=True)

chain.set_flags('validator',
                'unsafe-ws-external',
                'unsafe-rpc-external',
                'no-mdns',
                port=Seq(PORT),
                ws_port=Seq(WS_PORT),
                rpc_port=Seq(RPC_PORT),
                validator_port=Seq(VAL_PORT),
                public_validator_addresses=val_addrs,
                unit_creation_delay=500,
                execution='Native',
                rpc_cors='all',
                rpc_methods='Unsafe',
                state_pruning='archive'
               )

addresses = [n.address() for n in chain]
chain.set_flags(bootnodes=''.join(addresses), public_addr=addresses)

In [ ]:
# Helper function to upgrade nodes
def upgrade_node(i, purge=False, wait=True):
    prev = chain[i].change_binary(newbin, 'new', purge)
    time.sleep(5)
    if wait:
        try:
            chain.wait_for_finalization(prev, nodes=[i], timeout=120, finalized_delta=7)
            print(f'Node {i} finalization restored')
        except TimeoutError:
            print(f'Node {i} finalization stuck')
    chain.status()

In [ ]:
chain.start('old')
chain.wait_for_finalization(0)
chain.status()

In [ ]:
upgrade_node(0, wait=False)

In [ ]:
upgrade_node(1, purge=True)

In [ ]:
upgrade_node(2)

In [ ]:
upgrade_node(3)

In [ ]:
upgrade_node(4)

In [ ]:
upgrade_node(5)

In [ ]:
upgrade_node(6)

In [ ]:
chain[2].update_runtime(runtime, phrases[0])

In [ ]:
chain.stop()

In [ ]:
# TOOLBOX

In [ ]:
# General chain status
chain.status()

In [ ]:
# Restart some nodes
chain.stop(nodes=[2,3])
time.sleep(5)
chain.start(nodes=[2,3], 'restart')

In [ ]:
# Restart the whole chain
chain.stop()
time.sleep(5)
chain.start('fullrestart')

In [ ]:
# Change log level for one node (on the fly)
chain[6].set_log_level('sync', 'debug')

In [ ]:
# Change log level for all nodes (on the fly)
chain.set_log_level('sync', 'debug')

In [ ]:
# Grep node's current log for regexp
chain[0].greplog(r'best: #\d+ .+ finalized #\d+')